In [18]:
# Import library yang diperlukan
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import hog
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import os
from tqdm import tqdm

In [3]:
# Fungsi untuk memuat gambar dan mengekstrak fitur HOG
def load_and_extract_features(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (64, 64))  # Resize ke ukuran standar
    features, _ = hog(image, orientations=8, pixels_per_cell=(8, 8),
                      cells_per_block=(2, 2), visualize=True)
    return features

In [4]:
# Fungsi untuk memuat dataset
def load_dataset(data_path):
    features = []
    labels = []
    for label in os.listdir(data_path):
        label_path = os.path.join(data_path, label)
        if os.path.isdir(label_path):
            for image_file in tqdm(os.listdir(label_path), desc=f"Loading {label}"):
                image_path = os.path.join(label_path, image_file)
                features.append(load_and_extract_features(image_path))
                labels.append(label)
    return np.array(features), np.array(labels)

In [8]:
# Path ke dataset Anda
data_path = r"C:\Local Disk E\Code\Python\PCV\PokerCard\dataset"

# Memuat dataset
print("Loading dataset...")
features, labels = load_dataset(data_path)

Loading dataset...


In [10]:
def extract_hog_features(image_path):
    # Load image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # Resize the image (customize size as needed)
    image = cv2.resize(image, (64, 128))
    # Extract HOG features
    features, hog_image = hog(image, orientations=9, pixels_per_cell=(8, 8),
                              cells_per_block=(2, 2), visualize=True)
    return features

In [19]:
features = extract_hog_features(r'C:\Local Disk E\Code\Python\PCV\PokerCard\dataset\KS.png')

In [12]:
# Prepare dataset
features_list = []  # List of extracted HOG features
labels_list = []    # Corresponding labels

In [16]:
from sklearn.metrics import accuracy_score

In [17]:
# Loop over each image and extract HOG features, then label
for label, folder in enumerate(['sekop', 'hati', 'wajik', 'keriting']):
    for image_file in os.listdir(r'C:\Local Disk E\Code\Python\PCV\PokerCard\dataset\{}'.format(folder)):
        features = extract_hog_features(r'C:\Local Disk E\Code\Python\PCV\PokerCard\dataset\{}\{}'.format(folder, image_file))
        features_list.append(features)
        labels_list.append(label)

# Convert lists to numpy arrays
X = np.array(features_list)
y = np.array(labels_list)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train SVM model
clf = svm.SVC()
clf.fit(X_train, y_train)

# Evaluate model
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

Model Accuracy: 0.00%


### TEMPLATE MATCHING

In [21]:
import cv2
import numpy as np

def match_card(input_image_path, template_image_path):
    # Baca input gambar dan template
    input_image = cv2.imread(input_image_path, 0)
    template_image = cv2.imread(template_image_path, 0)

    # Terapkan template matching
    result = cv2.matchTemplate(input_image, template_image, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

    # Threshold untuk menentukan kecocokan
    threshold = 0.8
    if max_val >= threshold:
        print(f"Match found with score: {max_val}")
        # Gambarkan rectangle di sekitar area yang cocok
        h, w = template_image.shape
        top_left = max_loc
        bottom_right = (top_left[0] + w, top_left[1] + h)
        cv2.rectangle(input_image, top_left, bottom_right, (255, 0, 0), 2)
        cv2.imshow("Detected Card", input_image)
        cv2.waitKey(0)
    else:
        print("No match found.")

# Contoh penggunaan
match_card(r'C:\Local Disk E\Code\Python\PCV\PokerCard\dataset\3C.png', r'C:\Local Disk E\Code\Python\PCV\PokerCard\dataset\3C.png')


Match found with score: 1.0


### RESNET

In [24]:
import os
import json

def extract_labels(file_name):
    card_value = file_name[:-6]  # Mengambil nilai kartu
    card_suit = file_name[-6]    # Mengambil jenis kartu (S, D, H, C)
    return card_value, card_suit

# Baca file JSON
with open(r"C:\Users\agusf\Downloads\Compressed\annotation.json\annotation.json", 'r') as f:
    data = json.load(f)

labels = []
for image in data['images']:
    file_name = image['file_name']
    card_value, card_suit = extract_labels(file_name)
    labels.append({'file_name': file_name, 'value': card_value, 'suit': card_suit})

# Cetak label untuk setiap gambar
for label in labels:
    print(label)


{'file_name': '10D17.jpg', 'value': '10D', 'suit': '1'}
{'file_name': '10S18.jpg', 'value': '10S', 'suit': '1'}
{'file_name': 'KD49.jpg', 'value': 'KD', 'suit': '4'}
{'file_name': '2C4.jpg', 'value': '2', 'suit': 'C'}
{'file_name': 'AS38.jpg', 'value': 'AS', 'suit': '3'}
{'file_name': '8S36.jpg', 'value': '8S', 'suit': '3'}
{'file_name': '10D37.jpg', 'value': '10D', 'suit': '3'}
{'file_name': '3H43.jpg', 'value': '3H', 'suit': '4'}
{'file_name': '3D18.jpg', 'value': '3D', 'suit': '1'}
{'file_name': '10H32.jpg', 'value': '10H', 'suit': '3'}
{'file_name': 'QS18.jpg', 'value': 'QS', 'suit': '1'}
{'file_name': '7D1.jpg', 'value': '7', 'suit': 'D'}
{'file_name': '2D25.jpg', 'value': '2D', 'suit': '2'}
{'file_name': '9S28.jpg', 'value': '9S', 'suit': '2'}
{'file_name': 'QD46.jpg', 'value': 'QD', 'suit': '4'}
{'file_name': '4D21.jpg', 'value': '4D', 'suit': '2'}
{'file_name': '6H2.jpg', 'value': '6', 'suit': 'H'}
{'file_name': 'KD33.jpg', 'value': 'KD', 'suit': '3'}
{'file_name': '6D48.jpg', 

In [25]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load model ResNet50 pre-trained
base_model = ResNet50(weights='imagenet', include_top=False)

# Tambahkan layer klasifikasi di atasnya
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(52, activation='softmax')(x)  # 52 jenis kartu poker

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze semua layer dari ResNet50
for layer in base_model.layers:
    layer.trainable = False

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation dan preprocessing gambar
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    'path/to/dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    'path/to/dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Melatih model
model.fit(train_generator, validation_data=validation_generator, epochs=10)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 599s 6us/step


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'path/to/dataset'

: 